In [1]:
import pandas as pd
import csv
import re
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
import gensim
from gensim import corpora, models
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import time
from pprint import pprint

stemmer = SnowballStemmer("english") # the stopwords 
def remain_english(line):
    rule =re.compile("[^a-z^A-Z]")
    line = rule.sub('',line)
    return line
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v')) # remain problems

# Tokenize and lemmatize
def preprocess(text):
    result=[]
    for token in gensim.utils.simple_preprocess(str(text)) :
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
#             result.append(lemmatize_stemming(token))
            result.append(remain_english(lemmatize_stemming((token))).lower()) # take out not english and return into verb, and lower it 
    return result

c:\users\administrator\appdata\local\programs\python\python37\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# read file into pandas
import pandas as pd
fileGP = r'C:\Users\Administrator\Desktop\reviewData\Reviews(GP)\EducationReview.csv'
fileApple = r'C:\Users\Administrator\Desktop\reviewData\Reviews(Ap)\apple-education-reviews.csv'
# file = r'C:\Users\Administrator\Desktop\reviewData\dist\lifestyle-dist.csv'
df1 = pd.read_csv(fileGP)
df2 = pd.read_csv(fileApple)

In [3]:
gpNum = df1.count()['text']
ApNum = df2.count()['text'] # text(reviews) number from apple
sid = SentimentIntensityAnalyzer()
text_num = gpNum + ApNum
# text_num = 5500
documentNeg=[]
documentPos=[]
inconsistantNum1 = 0 # inconsistant reviews number in low rating reviews
inconsistantNum2 = 0
for i in range(gpNum):
    if(df1.score[i]<3): # low rating reviews
#         print(sid.polarity_scores(df1.text[i])['neg'])
        if(sid.polarity_scores(str(df1.text[i]))['pos'] > 0.5): # 如果是这条评论的sensitive分析分数显示这条评论其实是负面的,threhold = 0.5
            inconsistantNum1 += 1
        else:
            documentNeg.append(df1.text[i])             

    if(df1.score[i]>3): # high rating reviews
        if(sid.polarity_scores(str(df1.text[i]))['neg'] > 0.5): # if the neg score is high, not use this review,threhold = 0.5
            inconsistantNum2 += 1
        else:
            documentPos.append(df1.text[i]) 
            
for j in range(ApNum):
    if(df2.score[j]<3): # low rating reviews
        if(sid.polarity_scores(str(df2.text[j]))['pos'] > 0.5): # 如果是这条评论的sensitive分析分数显示这条评论其实是负面的,threhold = 0.5
            inconsistantNum1 += 1
        else:
            documentNeg.append(df2.text[j])             
    if(df2.score[j]>3): # low rating reviews
        if(sid.polarity_scores(str(df2.text[j]))['neg'] > 0.5): # if the neg score is high, not use this review,threhold = 0.5
            inconsistantNum2 += 1
        else:
            documentPos.append(df2.text[j])      
                        
print('number of reviews text:',text_num)
print('number of neg:',len(documentNeg))
print('number of inconsistant neg reviews text(actually a pos reviews):',inconsistantNum1)
print('number of pos:',len(documentPos))
print('number of inconsistant pos reviews text(actually an neg reviews):',inconsistantNum2)

number of reviews text: 219999
number of neg: 36006
number of inconsistant neg reviews text(actually a pos reviews): 350
number of pos: 169155
number of inconsistant pos reviews text(actually an neg reviews): 208


In [4]:
# negtive LDA  reviews analyse
document = documentNeg
doc_clean = [preprocess(doc) for doc in document]

dictionary = corpora.Dictionary(doc_clean)

# 使用上面的词典，将转换文档列表（语料）变成 DT 矩阵
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
print('made doc_term_matrix')

Lda = gensim.models.ldamodel.LdaModel
print('object num',len(doc_term_matrix))
start = time.time()
lda_model = gensim.models.LdaMulticore(doc_term_matrix, num_topics=10, id2word=dictionary, passes=100, workers=2)
# result = lda_model.print_topics(num_words=10)
end = time.time()
print ('Fitting Time:',end-start)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
# print(result)

made doc_term_matrix
object num 36006
Fitting Time: 2518.8920724391937
Topic: 0 
Words: 0.044*"work" + 0.042*"updat" + 0.038*"time" + 0.034*"crash" + 0.027*"open" + 0.018*"tri" + 0.017*"load" + 0.016*"download" + 0.014*"screen" + 0.011*"delet"
Topic: 1 
Words: 0.177*"" + 0.128*"version" + 0.063*"purchas" + 0.061*"sound" + 0.040*"upgrad" + 0.028*"letter" + 0.021*"buy" + 0.015*"code" + 0.014*"pay" + 0.011*"restor"
Topic: 2 
Words: 0.075*"game" + 0.035*"free" + 0.033*"money" + 0.031*"play" + 0.031*"like" + 0.023*"want" + 0.022*"wast" + 0.018*"time" + 0.017*"think" + 0.014*"download"
Topic: 3 
Words: 0.034*"month" + 0.026*"subscript" + 0.021*"pay" + 0.021*"free" + 0.020*"charg" + 0.017*"email" + 0.016*"account" + 0.015*"purchas" + 0.015*"cancel" + 0.014*"year"
Topic: 4 
Words: 0.029*"grade" + 0.017*"class" + 0.016*"book" + 0.015*"student" + 0.014*"need" + 0.014*"school" + 0.012*"like" + 0.011*"read" + 0.010*"better" + 0.010*"card"
Topic: 5 
Words: 0.041*"question" + 0.036*"answer" + 0.032*

In [5]:
# positive LDA reviews analyse
document = documentPos
doc_clean = [preprocess(doc) for doc in document]

dictionary = corpora.Dictionary(doc_clean)
# 使用上面的词典，将转换文档列表（语料）变成 DT 矩阵
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]
Lda = gensim.models.ldamodel.LdaModel
print('object num',len(doc_term_matrix))
start = time.time()
lda_model = gensim.models.LdaMulticore(doc_term_matrix, num_topics=10, id2word=dictionary, passes=100, workers=2)
# result = lda_model.print_topics(num_words=10)
end = time.time()
print ('Fitting Time:',end-start)
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))
# print(result)

object num 169155
Fitting Time: 8587.309166193008
Topic: 0 
Words: 0.067*"help" + 0.037*"math" + 0.036*"school" + 0.033*"grade" + 0.032*"student" + 0.026*"class" + 0.025*"love" + 0.021*"teacher" + 0.019*"great" + 0.019*"work"
Topic: 1 
Words: 0.023*"like" + 0.023*"updat" + 0.018*"work" + 0.016*"star" + 0.013*"abl" + 0.013*"option" + 0.013*"time" + 0.012*"featur" + 0.011*"need" + 0.011*"thing"
Topic: 2 
Words: 0.138*"good" + 0.054*"best" + 0.048*"free" + 0.024*"pretti" + 0.024*"app" + 0.024*"like" + 0.016*"better" + 0.014*"version" + 0.013*"think" + 0.012*"graph"
Topic: 3 
Words: 0.111*"game" + 0.053*"play" + 0.053*"love" + 0.046*"like" + 0.021*"think" + 0.021*"toca" + 0.014*"littl" + 0.013*"thing" + 0.012*"color" + 0.011*"want"
Topic: 4 
Words: 0.103*"love" + 0.055*"year" + 0.037*"kid" + 0.024*"daughter" + 0.023*"teach" + 0.022*"great" + 0.021*"learn" + 0.019*"worth" + 0.018*"read" + 0.018*"educ"
Topic: 5 
Words: 0.079*"great" + 0.034*"learn" + 0.023*"cours" + 0.020*"video" + 0.019*"le

In [ ]:
# try anlanyse with tfidf(Pos)
tfidf = models.TfidfModel(doc_term_matrix)
corpus_tfidf = tfidf[doc_term_matrix]
start = time.time()
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=100, workers=2)
end = time.time()
print ('Fitting Time:',end-start)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

In [ ]:
# try anlanyse with tfidf(Neg)
document = documentNeg
doc_clean = [preprocess(doc) for doc in document]

dictionary = corpora.Dictionary(doc_clean)

tfidf = models.TfidfModel(doc_term_matrix)
corpus_tfidf = tfidf[doc_term_matrix]
start = time.time()
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=100, workers=2)
end = time.time()
print ('Fitting Time:',end-start)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))